In [19]:
%matplotlib inline
from commons.config import DUMPED_MODEL

from IPython.core.display import display
from PIL import Image
from commons.utils import load_model
from ai.dnn import FashionModel, ColorModel, PoolingModel
from sklearn.metrics.pairwise import cosine_similarity
from feature_extractor import FeatureExtractor
from commons.image_utils import img_to_input
import pandas as pd

IMAGE_BASE_PATH = '/run/media/twoaday/data-storag/data-sets/multi-view-clothing/images/'

In [2]:
fashion_model = FashionModel(freeze_param=True, model_path=DUMPED_MODEL).cuda()
color_model = ColorModel().cuda()
pooling_model = PoolingModel().cuda()

In [3]:
extractor = FeatureExtractor(fashion_model, color_model, pooling_model)

In [13]:
df_triplets_medium = pd.read_csv('/run/media/twoaday/data-storag/data-sets/multi-view-clothing/triplets-medium.csv')

In [14]:
df_triplets_medium.count()

anchor      252162
positive    252162
negative    252162
dtype: int64

In [21]:
sample = [[row['anchor'], row['positive'], row['negative']] for _, row in df_triplets_medium.sample(1).iterrows()][0]
anchor_vec = extractor(img_to_input(IMAGE_BASE_PATH + sample[0]))

for name, img_id in zip(['anchor', 'positive', 'negative'], sample):
    path = IMAGE_BASE_PATH + img_id
    img = Image.open(path)
    img = img.resize((128, 128))
    vec = extractor(img_to_input(path))
    display(img)
    print('{0} : {1}'.format(name, cosine_similarity([anchor_vec, vec])))

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [64, 3, 7, 7], but got 3-dimensional input of size [3, 224, 224] instead